# Document Capsule

### Learning the language as is. Sequence TF IDf for Text Feature generation.

In [1]:
import numpy as np
import numba
import abc
from gensim.utils import simple_tokenize,simple_preprocess
import warnings
import re
import string
import unicodedata

In [2]:
import numba

In [3]:
# setting up things
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# warnings.filterwarnings('ignore',category=DeprecationWarning)

In [4]:
# input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True, 
# preprocessor=None, tokenizer=None, analyzer=’word’, stop_words=None, 
# token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), max_df=1.0, min_df=1, 
# max_features=None, vocabulary=None, 
# binary=False, dtype=<class ‘numpy.float64’>, norm=’l2’, use_idf=True, smooth_idf=True, sublinear_tf=False

In [5]:
texts = ['What are we doing here? Just trying to solve a problem! We will use capsules.',
         'What is a capsule? Lets read about it.',
         'Capsules are the single vectors represenation of multiple vectors.']

In [6]:
# this is not good for us
list(simple_tokenize(texts[0]))
simple_preprocess(texts[0])

['What',
 'are',
 'we',
 'doing',
 'here',
 'Just',
 'trying',
 'to',
 'solve',
 'a',
 'problem',
 'We',
 'will',
 'use',
 'capsules']

['what',
 'are',
 'we',
 'doing',
 'here',
 'just',
 'trying',
 'to',
 'solve',
 'problem',
 'we',
 'will',
 'use',
 'capsules']

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
# punct_re = re.compile('\?|\!|\.',re.UNICODE)
punct_re = re.compile('[?!.]')
punct_re.split(texts[0])
[i.split() for i  in punct_re.split(texts[0])]
texts[0].split().index('are')

['What are we doing here',
 ' Just trying to solve a problem',
 ' We will use capsules',
 '']

[['What', 'are', 'we', 'doing', 'here'],
 ['Just', 'trying', 'to', 'solve', 'a', 'problem'],
 ['We', 'will', 'use', 'capsules'],
 []]

1

In [38]:
def get_vocabulary(texts):
    vocab = []
    for text in texts:
        for token in simple_preprocess(text):
            vocab.append(token)
    return np.unique(vocab).tolist()
get_vocabulary(texts)
vocab = get_vocabulary(texts)
vocab.index('vectors')
len(vocab)
dict_vocab = dict([(k,i) for i,k in enumerate(vocab)])

['about',
 'are',
 'capsule',
 'capsules',
 'doing',
 'here',
 'is',
 'it',
 'just',
 'lets',
 'multiple',
 'of',
 'problem',
 'read',
 'represenation',
 'single',
 'solve',
 'the',
 'to',
 'trying',
 'use',
 'vectors',
 'we',
 'what',
 'will']

21

25

In [10]:
def pad_sentences(text,length,window):
    if isinstance(text,list):
        text_tokens = text
    else:
        text_tokens = text.split()
    if len(text_tokens) <= length:
        return text
    else:
        window_index = length - window
        prev_index=0
        texts = []
        while True:
            texts.append(text_tokens[prev_index:(prev_index+length)])
            prev_index = prev_index+length-window
            if prev_index >len(text_tokens):
                break
        
        return [text for text in texts if len(text) != 0]
    
pad_sentences(simple_preprocess(texts[0]),6,3)
        
        

[['what', 'are', 'we', 'doing', 'here', 'just'],
 ['doing', 'here', 'just', 'trying', 'to', 'solve'],
 ['trying', 'to', 'solve', 'problem', 'we', 'will'],
 ['problem', 'we', 'will', 'use', 'capsules'],
 ['use', 'capsules']]

In [11]:
simple_preprocess('')

[]

In [46]:
def generate_sequence_tf(texts, max_len, punct_re, preprocess =simple_preprocess, vocabulary=None,window=0):
    tf_vec = np.zeros((len(vocabulary),max_len))
    vocab_is_dict = isinstance(vocabulary,dict)
    for text in texts:
        for split_text in punct_re.split(text):
            processed_text = preprocess(split_text)
            l = len(processed_text)

            if l <= max_len and l != 0:
                for seq, word in enumerate(processed_text):
                    indx = vocabulary[word] if vocab_is_dict else vocabulary.index(word)
                    tf_vec[indx,seq] += 1/l

            else: # need to check its behaviour
                for padded_text in pad_sentences(processed_text, max_len, window=window):
                    for seq, word in enumerate(padded_text):
                        indx = vocabulary.index(word)
                        tf_vec[indx,seq] += 1
                    
    return tf_vec


@numba.jit
def generate_sequence_tf_numba(texts, max_len, punct_re, preprocess =simple_preprocess, vocabulary=None,window=0):
    tf_vec = np.zeros((len(vocabulary),max_len))
    i=1
    vocab_is_dict = isinstance(vocabulary,dict)
    for text in texts:
        for split_text in punct_re.split(text):
            processed_text = preprocess(split_text)
            l = len(processed_text)

            if l <= max_len and l != 0:
                for seq, word in enumerate(processed_text):
                    indx = vocabulary[word] if vocab_is_dict else vocabulary.index(word)
                    tf_vec[indx,seq] += 1/l

            else: # need to check its behaviour
                for padded_text in pad_sentences(processed_text, max_len, window=window):
                    for seq, word in enumerate(padded_text):
                        indx = vocabulary.index(word)
                        tf_vec[indx,seq] += 1
#         print(i)
        i+=1
                    
    return tf_vec
                
generate_sequence_tf(texts, 10, punct_re, vocabulary=dict_vocab)
generate_sequence_tf(texts, 10, punct_re, vocabulary=dict_vocab).shape

array([[0.        , 0.        , 0.25      , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.31111111, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.33333333, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.11111111, 0.        , 0.        , 0.25      , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.2       , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.2       ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.33333333, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

(25, 10)

In [34]:
generate_sequence_tf_numba(texts, 10, punct_re, vocabulary=vocab)

1
2
3


array([[0.        , 0.        , 0.25      , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.31111111, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.33333333, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.11111111, 0.        , 0.        , 0.25      , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.2       , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.2       ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.33333333, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [53]:
vocab_arr = np.array(vocab)


In [55]:
%timeit vocab_arr == 'what'

1.44 µs ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [56]:
%timeit np.where(vocab_arr == 'what')

2.04 µs ± 51.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [58]:
find_index = np.vectorize(lambda x, y : y == x)
%timeit find_index(vocab_arr,'what')

24.4 µs ± 361 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [59]:
find_index = np.vectorize(lambda x, y : y == x)
%timeit find_index(vocab,'what')

27.5 µs ± 318 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [51]:
%timeit generate_sequence_tf_numba(texts, 10, punct_re, vocabulary=vocab)

230 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [52]:
%timeit generate_sequence_tf_numba(texts, 10, punct_re, vocabulary=dict_vocab)

219 µs ± 6.81 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [49]:
%timeit generate_sequence_tf(texts, 10, punct_re, vocabulary=vocab)

89.7 µs ± 762 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [50]:
%timeit generate_sequence_tf(texts, 10, punct_re, vocabulary=dict_vocab)

81.7 µs ± 553 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [133]:
np.unique([(1,0),(1,0),(1,1)],axis=0)

array([[1, 0],
       [1, 1]])

In [62]:
import cProfile

In [63]:
%prun generate_sequence_tf(texts, 10, punct_re, vocabulary=dict_vocab)

         226 function calls in 0.004 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.004    0.004 <ipython-input-46-59e98ab18601>:1(generate_sequence_tf)
        1    0.001    0.001    0.004    0.004 {built-in method builtins.exec}
        9    0.000    0.000    0.001    0.000 utils.py:283(simple_preprocess)
        9    0.000    0.000    0.000    0.000 utils.py:223(tokenize)
       41    0.000    0.000    0.000    0.000 utils.py:265(simple_tokenize)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core.multiarray.zeros}
        9    0.000    0.000    0.000    0.000 utils.py:305(<listcomp>)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 {method 'split' of '_sre.SRE_Pattern' objects}
        3    0.000    0.000    0.000    0.000 <ipython-input-10-20eebb3b12a0>:1(pad_sentences)
       45    0.000    0.000    

In [64]:
%prun generate_sequence_tf_numba(texts, 10, punct_re, vocabulary=dict_vocab)

         447 function calls (440 primitive calls) in 0.017 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.004    0.004    0.017    0.017 <string>:1(<module>)
      2/1    0.003    0.002    0.004    0.004 <ipython-input-46-59e98ab18601>:23(generate_sequence_tf_numba)
        7    0.002    0.000    0.009    0.001 dispatcher.py:420(typeof_pyval)
        4    0.002    0.000    0.004    0.001 abstract.py:54(__call__)
        7    0.001    0.000    0.007    0.001 typeof.py:22(typeof)
        5    0.001    0.000    0.002    0.000 typeof.py:35(typeof_impl)
        5    0.001    0.000    0.001    0.000 typeof.py:56(_typeof_buffer)
      6/4    0.001    0.000    0.001    0.000 abstract.py:110(__hash__)
        5    0.000    0.000    0.001    0.000 api.py:178(typeof)
        4    0.000    0.000    0.001    0.000 {method 'get' of 'dict' objects}
        4    0.000    0.000    0.002    0.000 abstract.py:42(_intern)
     

In [35]:
def generate_sequence_idf(texts, max_len, punct_re, preprocess = simple_preprocess, vocabulary=None, window=0):
    idf_vec = np.zeros((len(vocabulary),max_len))
    for text in texts:
        combinations = []
        for split_text in punct_re.split(text):
            processed_text = preprocess(split_text)
            l = len(processed_text)
            if l <= max_len and l != 0:
                combinations += [[vocabulary.index(word),seq] for seq, word in enumerate(processed_text)]
                
            else: # need to check its behaviour
                for padded_text in pad_sentences(processed_text, max_len, window=window):
                    combinations += [(vocabulary.index(word),seq) for seq, word in enumerate(padded_text)]
        combinations = np.unique(combinations,axis=0)
        idf_vec[combinations[:,0],combinations[:,1]] += 1 
    return idf_vec

@numba.jit
def generate_sequence_idf_numba(texts, max_len, punct_re, preprocess = simple_preprocess, vocabulary=None, window=0):
    idf_vec = np.zeros((len(vocabulary),max_len))
    for text in texts:
        combinations = []
        for split_text in punct_re.split(text):
            processed_text = preprocess(split_text)
            l = len(processed_text)
            if l <= max_len and l != 0:
                for seq, word in enumerate(processed_text):
                    combinations.append([vocabulary.index(word),seq])
#                 combinations += [[vocabulary.index(word),seq] for seq, word in enumerate(processed_text)]
                
            else: # need to check its behaviour
                for padded_text in pad_sentences(processed_text, max_len, window=window):
                    for seq, word in enumerate(processed_text):
                        combinations.append([vocabulary.index(word),seq])
        combinations = np.unique(combinations,axis=0)
        idf_vec[combinations[:,0],combinations[:,1]] += 1 
    return idf_vec


# generate_sequence_idf(texts, 10, punct_re, vocabulary=vocab)
# generate_sequence_idf(texts, 10, punct_re, vocabulary=vocab).shape
generate_sequence_idf_numba(texts, 10, punct_re, vocabulary=vocab)

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 

In [36]:
%timeit generate_sequence_idf_numba(texts, 10, punct_re, vocabulary=vocab)

421 µs ± 6.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
%timeit generate_sequence_idf(texts, 10, punct_re, vocabulary=vocab)

231 µs ± 2.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [154]:
inp =np.arange(0,1,0.1).reshape(2,5)
# print(inp)
# inp.sum(axis=0)
def squashing_func_A(vectors,eps=1e-5):
    s_squared_norm = np.sum(np.square(vectors))
#     print(s_squared_norm)
    scale = np.sqrt(s_squared_norm + eps)
    return (s_squared_norm/(1+s_squared_norm))*(vectors / scale)
# squashing_func_A(inp)
# squashing_func_A(inp).sum(axis=0)

def squashing_func_B(vectors, dim=-1, eps=1e-5,return_normed=False):
    norm2 = np.sum(np.square(vectors), axis=dim)
    coeff = (np.sqrt(norm2+eps) / (1.0 + norm2))
    if return_normed:
        return (np.sqrt(norm2+eps) / (1.0 + norm2)) * vectors
    return coeff * np.sum(vectors,axis=dim)
# squashing_func_B(inp,dim=0,return_normed=True)
squashing_func_B(inp,dim=0)
# squashing_func_B(inp)

array([0.200004  , 0.31080229, 0.4282458 , 0.54326407, 0.64992798])

In [156]:
vocab.index('are')
generate_sequence_tf(texts, 10, punct_re, vocabulary=vocab)[vocab.index('are'),:]
generate_sequence_idf(texts, 10, punct_re, vocabulary=vocab)[vocab.index('are'),:]

1

array([0.        , 0.64444444, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

array([0., 3., 0., 0., 0., 0., 0., 0., 0., 0.])

In [68]:

def generate_sequence_tf_idf(texts, max_len, punct_re, preprocess = simple_preprocess, vocabulary=None, window=0):
    idf_vec = np.zeros((len(vocabulary),max_len))
    tf_vec = np.zeros((len(vocabulary),max_len))
    vocab_is_dict = isinstance(vocabulary,dict)
    for text in texts:
        combinations = []
        for split_text in punct_re.split(text):
            processed_text = preprocess(split_text)
            l = len(processed_text)
            if l <= max_len and l != 0:
                for seq, word in enumerate(processed_text):
                    indx = vocabulary[word] if vocab_is_dict else vocabulary.index(word)
                    combinations.append([indx,seq])
                    tf_vec[indx,seq] += 1/l
                
            else: # need to check its behaviour
                for padded_text in pad_sentences(processed_text, max_len, window=window):
                    for seq, word in enumerate(processed_text):
                        indx = vocabulary[word] if vocab_is_dict else vocabulary.index(word)
                        combinations.append([indx,seq])
                        tf_vec[indx,seq] += 1/l
        
        combinations = np.unique(combinations,axis=0)
        idf_vec[combinations[:,0],combinations[:,1]] += 1 
    return tf_vec,idf_vec

@numba.jit
def generate_sequence_tf_idf_numba(texts, max_len, punct_re, preprocess = simple_preprocess, vocabulary=None, window=0):
    idf_vec = np.zeros((len(vocabulary),max_len))
    tf_vec = np.zeros((len(vocabulary),max_len))
    vocab_is_dict = isinstance(vocabulary,dict)
    for text in texts:
        combinations = []
        for split_text in punct_re.split(text):
            processed_text = preprocess(split_text)
            l = len(processed_text)
            if l <= max_len and l != 0:
                for seq, word in enumerate(processed_text):
                    indx = vocabulary[word] if vocab_is_dict else vocabulary.index(word)
                    combinations.append([indx,seq])
                    tf_vec[indx,seq] += 1/l
                
            else: # need to check its behaviour
                for padded_text in pad_sentences(processed_text, max_len, window=window):
                    for seq, word in enumerate(processed_text):
                        indx = vocabulary[word] if vocab_is_dict else vocabulary.index(word)
                        combinations.append([indx,seq])
                        tf_vec[indx,seq] += 1/l
        
        combinations = np.unique(combinations,axis=0)
        idf_vec[combinations[:,0],combinations[:,1]] += 1 
    return tf_vec,idf_vec


In [71]:
%timeit generate_sequence_tf_idf_numba(texts, 10000, punct_re, vocabulary=vocab)

646 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [73]:
%timeit generate_sequence_tf_idf(texts, 10000, punct_re, vocabulary=vocab)

426 µs ± 3.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [165]:
def get_capsule_vectors(text, tf_vec, idf_vec, punct_re, num_docs, preprocess =simple_preprocess, vocabulary=None, window=0, return_normed=False):
    splits = punct_re.split(text)
    max_len = int(idf_vec.shape[1])
    capsules = np.zeros((len(splits),max_len))
    for cap_indx, split_text in enumerate(splits):
        processed_text = preprocess(split_text)
        l = len(processed_text)
        if l <= max_len and l != 0:
            for seq, word in enumerate(processed_text):
                indx = vocabulary.index(word)
                print(word, indx, seq, tf_vec[indx,seq], idf_vec[indx,seq])
                capsules[cap_indx,seq] += (tf_vec[indx,seq] * np.log(num_docs/idf_vec[indx,seq]) if idf_vec[indx,seq] != 0 else np.log(num_docs))
            
        else: # need to check its behaviour
            for padded_text in pad_sentences(processed_text, max_len, window=window):
                for seq, word in enumerate(padded_text):
                    indx = vocabulary.index(word)
                    capsules[cap_indx,seq] += tf_vec[indx,seq] * np.log(num_docs/idf_vec[indx,seq])
    print(capsules)            
    return squashing_func_B(capsules,dim=0,return_normed=return_normed)

get_capsule_vectors('What are capsules? Lets read about it.',
                    generate_sequence_tf(texts, 10, punct_re, vocabulary=vocab),
                    generate_sequence_idf(texts, 10, punct_re, vocabulary=vocab),
                    punct_re=punct_re, num_docs=3, vocabulary=vocab,return_normed=True)

get_capsule_vectors('What are capsules? Lets read about it.',
                    generate_sequence_tf(texts, 10, punct_re, vocabulary=vocab),
                    generate_sequence_idf(texts, 10, punct_re, vocabulary=vocab),
                    punct_re=punct_re, num_docs=3, vocabulary=vocab)

what 23 0 0.5333333333333333 2.0
are 1 1 0.3111111111111111 2.0
capsules 3 2 0.0 0.0
lets 9 0 0.25 1.0
read 13 1 0.25 1.0
about 0 2 0.25 1.0
it 7 3 0.25 1.0
[[0.21624806 0.1261447  1.09861229 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.27465307 0.27465307 0.27465307 0.27465307 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]]


array([[0.06736462, 0.03493628, 0.54508784, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.08555869, 0.07606627, 0.13627196, 0.07014776, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

what 23 0 0.5333333333333333 2.0
are 1 1 0.3111111111111111 2.0
capsules 3 2 0.0 0.0
lets 9 0 0.25 1.0
read 13 1 0.25 1.0
about 0 2 0.25 1.0
it 7 3 0.25 1.0
[[0.21624806 0.1261447  1.09861229 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.27465307 0.27465307 0.27465307 0.27465307 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]]


array([0.15292332, 0.11100255, 0.6813598 , 0.07014776, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [22]:
class DocumentCapsule():
    
    def __init__(self, feature_len=100, sequence_tf=True, sequence_idf=True, sentence_splitter='?!.',
                 pad_senetence_window=0, pad_value='global_0', compressor='squash', dictionary=None, tokenizer=None,
                 preprocess = simple_preprocess, stopwords=None, epsilon=1e-6,return_normed=False):
        
        self.feature_len = feature_len
        self.sequence_tf = sequence_tf
        self.sequence_idf = sequence_idf
        self.sentence_splitter = sentence_splitter
        self.pad_senetence_window = pad_senetence_window
        self.pad_value = pad_value
        self.compressor = 'squash'
        self.preprocess = preprocess
        self.dictionary = dictionary
        self.tokenizer = tokenizer
        self.stopwords = stopwords
        self.eps = epsilon
        self.return_normed = return_normed
        
        self.preprocessor = self.tokenizer if tokenizer is not None else self.preprocess
        self.tf_vectors = None
        self.idf_vectors = None
        self.num_docs = None
        self.vocabulary = None
        
        self.sentence_splitter_re = re.compile('\?|\!|\.',re.UNICODE)
        
    def _squash(self, vectors, dim=-1, return_normed=False):
        norm2 = np.sum(np.square(vectors), axis=dim)
        coeff = (np.sqrt(norm2+self.eps) / (1.0 + norm2))
        if return_normed:
            return (np.sqrt(norm2+eps) / (1.0 + norm2)) * vectors
        return coeff * np.sum(vectors,axis=dim)
    
    @numba.jit
    def _generate_sequence_tf(self, texts):
        
        preprocess= self.preprocessor
        max_len = self.feature_len
        punct_re = self.sentence_splitter_re
        vocabulary = self.vocabulary
        window = self.pad_senetence_window
        
        tf_vec = np.zeros((len(vocabulary),max_len))
        for text in texts:
            for split_text in punct_re.split(text):
                processed_text = preprocess(split_text)
                l = len(processed_text)

                if l <= max_len and l != 0:
                    for seq, word in enumerate(processed_text):
                        indx = vocabulary.index(word)
                        tf_vec[indx,seq] += 1/l

                else: # need to check its behaviour
                    for padded_text in pad_sentences(processed_text, max_len, window=window):
                        for seq, word in enumerate(padded_text):
                            indx = vocabulary.index(word)
                            tf_vec[indx,seq] += 1

        return tf_vec
    
#     @numba.jit
    def _generate_sequence_idf(self, texts):
        
        preprocess= self.preprocessor
        max_len = self.feature_len
        punct_re = self.sentence_splitter_re
        vocabulary = self.vocabulary
        window = self.pad_senetence_window
        idf_vec = np.zeros((len(vocabulary),max_len))

        for text in texts:
            combinations = []
            for split_text in punct_re.split(text):
                processed_text = preprocess(split_text)
                l = len(processed_text)
                if l <= max_len and l != 0:
                    combinations += [[vocabulary.index(word),seq] for seq, word in enumerate(processed_text)]

                else: # need to check its behaviour
                    for padded_text in pad_sentences(processed_text, max_len, window=window):
                        combinations += [(vocabulary.index(word),seq) for seq, word in enumerate(padded_text)]
            combinations = np.unique(combinations,axis=0)
            idf_vec[combinations[:,0],combinations[:,1]] += 1 
        return idf_vec
    
    def _generate_vocabulary(self, texts):
        vocab = []
        for text in texts:
            for token in self.preprocessor(text):
                vocab.append(token)
        return np.unique(vocab).tolist()
    
    @numba.jit
    def _get_capsule_vectors(self, text):
        
        preprocess= self.preprocessor
        max_len = self.feature_len
        vocabulary = self.vocabulary
        window = self.pad_senetence_window
        return_normed = self.return_normed
        tf_vec = self.tf_vectors
        idf_vec = self.idf_vectors
        num_docs = self.num_docs 
        splits = self.sentence_splitter_re.split(text)
        capsules = np.zeros((len(splits),max_len))
        
        for cap_indx, split_text in enumerate(splits):
            processed_text = preprocess(split_text)
            l = len(processed_text)
            if l <= max_len and l != 0:
                for seq, word in enumerate(processed_text):
                    indx = vocabulary.index(word)
                    #print(word, indx, seq, tf_vec[indx,seq], idf_vec[indx,seq])
                    capsules[cap_indx,seq] += (tf_vec[indx,seq] * np.log(num_docs/idf_vec[indx,seq]) if idf_vec[indx,seq] != 0 else np.log(num_docs))

            else: # need to check its behaviour
                for padded_text in pad_sentences(processed_text, max_len, window=window):
                    for seq, word in enumerate(padded_text):
                        indx = vocabulary.index(word)
                        capsules[cap_indx,seq] += tf_vec[indx,seq] * np.log(num_docs/idf_vec[indx,seq])
        
        # print(capsules)            
        return self._squash(capsules, dim=0, return_normed=return_normed)
        
    def fit(self,X):
        
        if self.dictionary is None:
            self.vocabulary = self._generate_vocabulary(X)

        if self.sequence_tf:
            self.tf_vectors = self._generate_sequence_tf(X)
        else:
            self.tf_vectors= np.ones((len(self.vocabulary),self.feature_len))
        
        if self.sequence_idf:
            self.idf_vectors = self._generate_sequence_idf(X)
        else:
            self.idf_vectors = np.ones((len(self.vocabulary),self.feature_len))
            
        self.num_docs = len(X)
        
    def transform(self,X):
        num_docs_to_transform = len(X)
        transform_capsule = np.zeros((num_docs_to_transform,self.feature_len))
        for idx,text in enumerate(X):
            transform_capsule[idx,:] = self._get_capsule_vectors(text)
        return transform_capsule
            
            
            
        
        

In [23]:
doccapsule = DocumentCapsule()

In [24]:
doccapsule.fit(texts)

In [25]:
doccapsule.transform(['What are capsules? Lets read about it.'])

array([[0.15291769, 0.11099708, 0.68135741, 0.07014358, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

3